In [18]:
import paddle
import paddle.nn as nn
import paddle.nn.functional as F

In [36]:
class ConvBNLayer(nn.Layer):
    def __init__(self,
                ch_in,
                ch_out,
                kernel_size=1,
                stride=1,
                padding=0,
                groups=1,
                use_act=True):
        super(ConvBNLayer, self).__init__()
        self.use_act = use_act
        self.conv = nn.Conv2D(ch_in,
                            ch_out,
                            kernel_size=kernel_size,
                            stride=stride,
                            padding=padding)
        self.bn = nn.BatchNorm2D(ch_out)
        if use_act:
            self.act = nn.ReLU()
    def forward(self,x):
        x = self.conv(x)
        x = self.bn(x)
        if self.use_act:
            x = self.act(x)
        return x

In [43]:
class BottleBlock(nn.Layer):
    expansion = 4
    def __init__(self,
                ch_in,
                ch_out,
                stride=1,
                short_cut=True,
                use_ese=False):
        super(BottleBlock, self).__init__()
        self.conv1 = ConvBNLayer(
            ch_in,
            ch_out,
            kernel_size=1
        )
        self.conv2 = ConvBNLayer(
            ch_out,
            ch_out,
            kernel_size=3,
            stride=stride,
            padding=1
        )
        self.conv3 = ConvBNLayer(
            ch_out,
            ch_out*4,
            kernel_size=1,
        )
        self.short = nn.Sequential()
        if stride==2 or ch_in  != ch_out*self.expansion :
            self.short = ConvBNLayer(
                ch_in,
                ch_out*4,
                kernel_size=1,
                stride=stride,
            )
        self.short_cut = short_cut
    def forward(self,x):
        input = x
        x = self.conv1(x)
        x = self.conv2(x)
        x = self.conv3(x)
        if self.short_cut:
            x = x + self.short(input)
        else:
            x = x + input
        return x
        


In [54]:
class ResNet(nn.Layer):
    def __init__(self,
                in_channels=3,
                out_channels=1000,
                depths=[3,4,6,3],
                widths=[64,128,256,512],
                strides=[1,2,2,2]):
        super(ResNet, self).__init__()
        Block = BottleBlock

        self.stem = nn.Sequential(
            nn.Conv2D(in_channels,widths[0],kernel_size=4,stride=4),
            nn.BatchNorm2D(widths[0]),
            nn.ReLU())
        self.in_channels = 64
        self.conv1 = self._make_layer(Block, 64, depths[0], 1)
        self.conv2 = self._make_layer(Block, 128, depths[1], 2)
        self.conv3 = self._make_layer(Block, 256, depths[2], 2)
        self.conv4 = self._make_layer(Block, 512, depths[3], 2)
        self.avg_pool = nn.AdaptiveAvgPool2D((1, 1))
        self.fc = nn.Linear(512 * Block.expansion, out_channels)

    
    def _make_layer(self, block, out_channels, num_blocks, stride):
        strides = [stride]+[1]*(num_blocks-1)
        layers = []
        for stride in strides:
            layers.append(block(self.in_channels, out_channels, stride))
            self.in_channels = out_channels * 4
        return nn.Sequential(*layers)

    def forward(self, x):
        output = self.stem(x)
        output = self.conv1(output)
        output = self.conv2(output)
        output = self.conv3(output)
        output = self.conv4(output)
        output = self.avg_pool(output)
        output = output.reshape((output.shape[0], -1))
        output = self.fc(output)

        return output
        
        

In [55]:
model = ResNet()
paddle.summary(model,(1,3,224,224))

--------------------------------------------------------------------------------
    Layer (type)         Input Shape          Output Shape         Param #    
     Conv2D-690       [[1, 3, 224, 224]]    [1, 64, 56, 56]         3,136     
  BatchNorm2D-690     [[1, 64, 56, 56]]     [1, 64, 56, 56]          256      
      ReLU-383        [[1, 64, 56, 56]]     [1, 64, 56, 56]           0       
     Conv2D-691       [[1, 64, 56, 56]]     [1, 64, 56, 56]         4,160     
  BatchNorm2D-691     [[1, 64, 56, 56]]     [1, 64, 56, 56]          256      
      ReLU-384        [[1, 64, 56, 56]]     [1, 64, 56, 56]           0       
  ConvBNLayer-672     [[1, 64, 56, 56]]     [1, 64, 56, 56]           0       
     Conv2D-692       [[1, 64, 56, 56]]     [1, 64, 56, 56]        36,928     
  BatchNorm2D-692     [[1, 64, 56, 56]]     [1, 64, 56, 56]          256      
      ReLU-385        [[1, 64, 56, 56]]     [1, 64, 56, 56]           0       
  ConvBNLayer-673     [[1, 64, 56, 56]]     [1, 64

{'total_params': 25630376, 'trainable_params': 25524136}